In [3]:
import gym
import matplotlib.pyplot as plt
import time
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [6]:
env = gym.make("CartPole-v1", render_mode='human') # render_mode='rgb_array'

reset_graph()

n_inputs = 4
n_hidden = 4
n_outputs = 1

learning_rate = 0.01

initializer = tf.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs = tf.nn.sigmoid(logits)  # probability of action 0 (left)
p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

y = 1. - tf.to_float(action)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
num_steps = 1000
num_episodes = 10
totals = []
terminated_num = 0
truncated_num = 0
model_path = "my_policy_net_pg.ckpt"

for episode in range(num_episodes):
    episode_reward = 0
    obs = env.reset()[0]
    with tf.Session() as sess:
        saver.restore(sess, model_path)
        for step in range(num_steps):
            action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})

            # app={ly the action
            obs, reward, terminated, truncated, info  = env.step(action_val[0][0])
            episode_reward += reward
            ## Render the env
            #env.render()

            ## Wait a bit before the next frame unless you want to see a crazy fast video
            #time.sleep(0.001)

            # If the epsiode is up, then start another one
            if terminated:
                terminated_num += 1
                break
            if truncated:
                truncated_num += 1
                break
        totals.append(episode_reward)

print(f"Mean: {np.mean(totals)}; STD: {np.std(totals)}; Min: {np.min(totals)}; Max: {np.max(totals)}; Terminated: {terminated_num}; Truncated: {truncated_num}")

INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
INFO:tensorflow:Restoring parameters from my_policy_net_pg.ckpt
Mean: 238.3; STD: 117.97970164396925; Min: 125.0; Max: 500.0; Terminated: 9; Truncated: 1
